In [ ]:
import numpy as np
import pandas as pd
import warnings
import joblib
import os
import time
from tqdm import tqdm
from scipy.optimize import minimize
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.isotonic import IsotonicRegression
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import torch

warnings.filterwarnings("ignore")
np.random.seed(42)

# =============================================================================
# GPU CONFIGURATION
# =============================================================================
print("="*80)
print("META-LEARNER, ENSEMBLE & SUBMISSION")
print("="*80)
if torch.cuda.is_available():
    print(f"✓ GPU detected: {torch.cuda.get_device_name(0)}")
    USE_GPU = True
else:
    print("⚠ No GPU detected, using CPU")
    USE_GPU = False

# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../data/preprocessed/X_train.pkl')
X_test = joblib.load('../data/preprocessed/X_test.pkl')
X_orig = joblib.load('../data/preprocessed/X_orig.pkl')
y_train = joblib.load('../data/preprocessed/y_train.pkl')
y_orig = joblib.load('../data/preprocessed/y_orig.pkl')
metadata = joblib.load('../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Test shape: {X_test.shape}")
print(f"✓ Original shape: {X_orig.shape}")

# Load original test for ID column
test_df = pd.read_csv("../data/test.csv")

# =============================================================================
# LOAD OPTIMIZED PARAMETERS
# =============================================================================
print("\n" + "="*80)
print("LOADING OPTIMIZED PARAMETERS")
print("="*80)

params_lightgbm = joblib.load('../results/optimization/lightgbm_params.pkl')
params_xgboost = joblib.load('../results/optimization/xgboost_params.pkl')
params_catboost = joblib.load('../results/optimization/catboost_params.pkl')
params_extratrees = joblib.load('../results/optimization/extratrees_params.pkl')
params_gradientboosting = joblib.load('../results/optimization/gradientboosting_params.pkl')
params_ridge = joblib.load('../results/optimization/ridge_params.pkl')
params_elasticnet = joblib.load('../results/optimization/elasticnet_params.pkl')
params_svr = joblib.load('../results/optimization/svr_params.pkl')

print("✓ All 8 model parameters loaded")

# =============================================================================
# TRAINING CONFIGURATION
# =============================================================================
N_FOLDS = 10
y_bins = pd.qcut(y_train, q=10, labels=False, duplicates='drop')
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

print(f"\n{'='*60}")
print(f"TRAINING CONFIGURATION")
print(f"{'='*60}")
print(f"Folds: {N_FOLDS}")
print(f"Strategy: Stratified K-Fold")

# Storage for OOF and test predictions
oof_predictions = {
    'ridge': np.zeros(len(y_train)),
    'elastic': np.zeros(len(y_train)),
    'xgb': np.zeros(len(y_train)),
    'lgb': np.zeros(len(y_train)),
    'cat': np.zeros(len(y_train)),
    'et': np.zeros(len(y_train)),
    'gbr': np.zeros(len(y_train)),
    'svr': np.zeros(len(y_train))
}

test_predictions = {
    'ridge': np.zeros((len(X_test), N_FOLDS)),
    'elastic': np.zeros((len(X_test), N_FOLDS)),
    'xgb': np.zeros((len(X_test), N_FOLDS)),
    'lgb': np.zeros((len(X_test), N_FOLDS)),
    'cat': np.zeros((len(X_test), N_FOLDS)),
    'et': np.zeros((len(X_test), N_FOLDS)),
    'gbr': np.zeros((len(X_test), N_FOLDS)),
    'svr': np.zeros((len(X_test), N_FOLDS))
}

best_iterations = {
    'xgb': [],
    'lgb': [],
    'cat': []
}

# Storage for full models
full_models = {}

# =============================================================================
# LEVEL 1: BASE MODELS TRAINING
# =============================================================================
print("\n" + "="*80)
print("LEVEL 1: BASE MODELS TRAINING (8 MODELS)")
print("="*80)

for fold, (train_idx, val_idx) in enumerate(tqdm(skf.split(X_train, y_bins), 
                                                   total=N_FOLDS, 
                                                   desc="Training Folds"), 1):
    print(f"\n{'='*60}")
    print(f"FOLD {fold}/{N_FOLDS}")
    print(f"{'='*60}")

    # Split data
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]

    # Augment with original data
    X_tr_full = pd.concat([X_tr, X_orig], axis=0, ignore_index=True)
    y_tr_full = np.concatenate([y_tr, y_orig])

    print(f"Training size: {len(X_tr_full):,} ({len(X_tr):,} synthetic + {len(X_orig):,} original)")

    # --- RIDGE ---
    print("Ridge...", end=" ", flush=True)
    scaler_ridge = RobustScaler()
    X_tr_scaled = scaler_ridge.fit_transform(X_tr_full)
    X_val_scaled = scaler_ridge.transform(X_val)
    X_test_scaled = scaler_ridge.transform(X_test)

    ridge = Ridge(**params_ridge)
    ridge.fit(X_tr_scaled, y_tr_full)
    oof_predictions['ridge'][val_idx] = np.clip(ridge.predict(X_val_scaled), 0, 100)
    test_predictions['ridge'][:, fold-1] = np.clip(ridge.predict(X_test_scaled), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['ridge'][val_idx])):.5f}")

    # --- ELASTICNET ---
    print("ElasticNet...", end=" ", flush=True)
    elastic = ElasticNet(**params_elasticnet)
    elastic.fit(X_tr_scaled, y_tr_full)
    oof_predictions['elastic'][val_idx] = np.clip(elastic.predict(X_val_scaled), 0, 100)
    test_predictions['elastic'][:, fold-1] = np.clip(elastic.predict(X_test_scaled), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['elastic'][val_idx])):.5f}")

    # --- XGBOOST ---
    print("XGBoost...", end=" ", flush=True)
    dtrain = xgb.DMatrix(X_tr_full, label=y_tr_full)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)

    xgb_model = xgb.train(
        params_xgboost,
        dtrain,
        num_boost_round=3000,
        evals=[(dval, 'eval')],
        early_stopping_rounds=100,
        verbose_eval=False
    )

    best_iterations['xgb'].append(xgb_model.best_iteration)
    oof_predictions['xgb'][val_idx] = np.clip(xgb_model.predict(dval), 0, 100)
    test_predictions['xgb'][:, fold-1] = np.clip(xgb_model.predict(dtest), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['xgb'][val_idx])):.5f} (iter: {xgb_model.best_iteration})")

    # --- LIGHTGBM ---
    print("LightGBM...", end=" ", flush=True)
    train_data = lgb.Dataset(X_tr_full, label=y_tr_full)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    lgb_model = lgb.train(
        params_lightgbm,
        train_data,
        num_boost_round=3000,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )

    best_iterations['lgb'].append(lgb_model.best_iteration)
    oof_predictions['lgb'][val_idx] = np.clip(lgb_model.predict(X_val), 0, 100)
    test_predictions['lgb'][:, fold-1] = np.clip(lgb_model.predict(X_test), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['lgb'][val_idx])):.5f} (iter: {lgb_model.best_iteration})")

    # --- CATBOOST ---
    print("CatBoost...", end=" ", flush=True)
    cat_model = CatBoostRegressor(**params_catboost)
    cat_model.fit(X_tr_full, y_tr_full, eval_set=(X_val, y_val), verbose=False)

    best_iterations['cat'].append(cat_model.get_best_iteration())
    oof_predictions['cat'][val_idx] = np.clip(cat_model.predict(X_val), 0, 100)
    test_predictions['cat'][:, fold-1] = np.clip(cat_model.predict(X_test), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['cat'][val_idx])):.5f} (iter: {cat_model.get_best_iteration()})")

    # --- EXTRATREES ---
    print("ExtraTrees...", end=" ", flush=True)
    et_model = ExtraTreesRegressor(**params_extratrees)
    et_model.fit(X_tr_full, y_tr_full)
    oof_predictions['et'][val_idx] = np.clip(et_model.predict(X_val), 0, 100)
    test_predictions['et'][:, fold-1] = np.clip(et_model.predict(X_test), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['et'][val_idx])):.5f}")

    # --- GRADIENT BOOSTING ---
    print("GradientBoosting...", end=" ", flush=True)
    gbr_model = GradientBoostingRegressor(**params_gradientboosting)
    gbr_model.fit(X_tr_full, y_tr_full)
    oof_predictions['gbr'][val_idx] = np.clip(gbr_model.predict(X_val), 0, 100)
    test_predictions['gbr'][:, fold-1] = np.clip(gbr_model.predict(X_test), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['gbr'][val_idx])):.5f}")

    # --- SVR ---
    print("SVR...", end=" ", flush=True)
    sample_size = min(100000, len(X_tr_full))
    if len(X_tr_full) > sample_size:
        idx = np.random.choice(len(X_tr_full), sample_size, replace=False)
        X_tr_sample = X_tr_full.iloc[idx]
        y_tr_sample = y_tr_full[idx]
    else:
        X_tr_sample = X_tr_full
        y_tr_sample = y_tr_full

    scaler_svr = StandardScaler()
    X_tr_svr = scaler_svr.fit_transform(X_tr_sample)
    X_val_svr = scaler_svr.transform(X_val)
    X_test_svr = scaler_svr.transform(X_test)

    svr_model = SVR(**params_svr)
    svr_model.fit(X_tr_svr, y_tr_sample)
    oof_predictions['svr'][val_idx] = np.clip(svr_model.predict(X_val_svr), 0, 100)
    test_predictions['svr'][:, fold-1] = np.clip(svr_model.predict(X_test_svr), 0, 100)
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val, oof_predictions['svr'][val_idx])):.5f}")

# Level 1 OOF scores
print(f"\n{'='*80}")
print("LEVEL 1 OOF SCORES")
print(f"{'='*80}")
for model_name, oof in oof_predictions.items():
    rmse = np.sqrt(mean_squared_error(y_train, oof))
    print(f"{model_name.upper():15s}: {rmse:.6f}")

# =============================================================================
# FULL REFIT ON ALL DATA
# =============================================================================
print(f"\n{'='*80}")
print("FULL REFIT: TRAINING ON 100% OF DATA")
print(f"{'='*80}")

avg_best_iter_xgb = int(np.mean(best_iterations['xgb']))
avg_best_iter_lgb = int(np.mean(best_iterations['lgb']))
avg_best_iter_cat = int(np.mean(best_iterations['cat']))

print(f"\nAverage best iterations from CV:")
print(f"  XGBoost:  {avg_best_iter_xgb}")
print(f"  LightGBM: {avg_best_iter_lgb}")
print(f"  CatBoost: {avg_best_iter_cat}")

# Prepare full dataset
X_full = pd.concat([X_train, X_orig], axis=0, ignore_index=True)
y_full = np.concatenate([y_train, y_orig])

print(f"\nFull training set size: {len(X_full):,} samples")

# Storage for full model predictions
test_pred_full = {}

print("\nTraining final models on 100% data...")

# Ridge
print("  Ridge...", end=" ", flush=True)
scaler_full = RobustScaler()
X_full_scaled = scaler_full.fit_transform(X_full)
X_test_full_scaled = scaler_full.transform(X_test)

ridge_full = Ridge(**params_ridge)
ridge_full.fit(X_full_scaled, y_full)
test_pred_full['ridge'] = np.clip(ridge_full.predict(X_test_full_scaled), 0, 100)
full_models['ridge'] = {'model': ridge_full, 'scaler': scaler_full}
print("✓")

# ElasticNet
print("  ElasticNet...", end=" ", flush=True)
elastic_full = ElasticNet(**params_elasticnet)
elastic_full.fit(X_full_scaled, y_full)
test_pred_full['elastic'] = np.clip(elastic_full.predict(X_test_full_scaled), 0, 100)
full_models['elastic'] = {'model': elastic_full, 'scaler': scaler_full}
print("✓")

# XGBoost
print("  XGBoost...", end=" ", flush=True)
dtrain_full = xgb.DMatrix(X_full, label=y_full)
dtest_full = xgb.DMatrix(X_test)
xgb_full = xgb.train(params_xgboost, dtrain_full, num_boost_round=avg_best_iter_xgb, verbose_eval=False)
test_pred_full['xgb'] = np.clip(xgb_full.predict(dtest_full), 0, 100)
full_models['xgb'] = xgb_full
print("✓")

# LightGBM
print("  LightGBM...", end=" ", flush=True)
train_full_lgb = lgb.Dataset(X_full, label=y_full)
lgb_full = lgb.train(params_lightgbm, train_full_lgb, num_boost_round=avg_best_iter_lgb, callbacks=[lgb.log_evaluation(0)])
test_pred_full['lgb'] = np.clip(lgb_full.predict(X_test), 0, 100)
full_models['lgb'] = lgb_full
print("✓")

# CatBoost
print("  CatBoost...", end=" ", flush=True)
cat_params_full = params_catboost.copy()
cat_params_full['iterations'] = avg_best_iter_cat
cat_full = CatBoostRegressor(**cat_params_full)
cat_full.fit(X_full, y_full)
test_pred_full['cat'] = np.clip(cat_full.predict(X_test), 0, 100)
full_models['cat'] = cat_full
print("✓")

# ExtraTrees
print("  ExtraTrees...", end=" ", flush=True)
et_full = ExtraTreesRegressor(**params_extratrees)
et_full.fit(X_full, y_full)
test_pred_full['et'] = np.clip(et_full.predict(X_test), 0, 100)
full_models['et'] = et_full
print("✓")

# GradientBoosting
print("  GradientBoosting...", end=" ", flush=True)
gbr_full = GradientBoostingRegressor(**params_gradientboosting)
gbr_full.fit(X_full, y_full)
test_pred_full['gbr'] = np.clip(gbr_full.predict(X_test), 0, 100)
full_models['gbr'] = gbr_full
print("✓")

# SVR
print("  SVR...", end=" ", flush=True)
sample_size = min(100000, len(X_full))
if len(X_full) > sample_size:
    idx = np.random.choice(len(X_full), sample_size, replace=False)
    X_full_sample = X_full.iloc[idx]
    y_full_sample = y_full[idx]
else:
    X_full_sample = X_full
    y_full_sample = y_full

scaler_svr_full = StandardScaler()
X_full_svr = scaler_svr_full.fit_transform(X_full_sample)
X_test_svr = scaler_svr_full.transform(X_test)

svr_full = SVR(**params_svr)
svr_full.fit(X_full_svr, y_full_sample)
test_pred_full['svr'] = np.clip(svr_full.predict(X_test_svr), 0, 100)
full_models['svr'] = {'model': svr_full, 'scaler': scaler_svr_full}
print("✓")

print("\n✓ All models trained on full data!")

# Save full models
os.makedirs('../models', exist_ok=True)
for model_name, model_obj in full_models.items():
    joblib.dump(model_obj, f'../models/{model_name}_full.pkl')
print("\n✓ Full models saved!")

# =============================================================================
# LEVEL 2: META-LEARNER STACKING
# =============================================================================
print(f"\n{'='*80}")
print("LEVEL 2: META-LEARNER STACKING")
print(f"{'='*80}")

# Create meta-features
meta_train = np.column_stack(list(oof_predictions.values()))
meta_test = np.column_stack(list(test_pred_full.values()))

# Add statistical features
meta_train_enhanced = np.column_stack([
    meta_train,
    meta_train.mean(axis=1),
    meta_train.std(axis=1),
    meta_train.max(axis=1),
    meta_train.min(axis=1)
])

meta_test_enhanced = np.column_stack([
    meta_test,
    meta_test.mean(axis=1),
    meta_test.std(axis=1),
    meta_test.max(axis=1),
    meta_test.min(axis=1)
])

print(f"Meta-features shape: {meta_train_enhanced.shape}")

# Train meta-learners
oof_meta_ridge = np.zeros(len(y_train))
oof_meta_lgb = np.zeros(len(y_train))
oof_meta_xgb = np.zeros(len(y_train))

test_meta_ridge = np.zeros((len(X_test), N_FOLDS))
test_meta_lgb = np.zeros((len(X_test), N_FOLDS))
test_meta_xgb = np.zeros((len(X_test), N_FOLDS))

meta_models = {
    'ridge': [],
    'lgb': [],
    'xgb': [],
    'scalers': []
}

for fold, (train_idx, val_idx) in enumerate(skf.split(meta_train_enhanced, y_bins), 1):
    print(f"Meta-learner Fold {fold}/{N_FOLDS}...", end=" ")

    X_tr_meta, X_val_meta = meta_train_enhanced[train_idx], meta_train_enhanced[val_idx]
    y_tr_meta, y_val_meta = y_train[train_idx], y_train[val_idx]

    # Ridge meta
    scaler_meta = StandardScaler()
    X_tr_meta_scaled = scaler_meta.fit_transform(X_tr_meta)
    X_val_meta_scaled = scaler_meta.transform(X_val_meta)
    X_test_meta_scaled = scaler_meta.transform(meta_test_enhanced)

    ridge_meta = Ridge(alpha=1.0, random_state=42)
    ridge_meta.fit(X_tr_meta_scaled, y_tr_meta)
    oof_meta_ridge[val_idx] = np.clip(ridge_meta.predict(X_val_meta_scaled), 0, 100)
    test_meta_ridge[:, fold-1] = np.clip(ridge_meta.predict(X_test_meta_scaled), 0, 100)
    
    meta_models['ridge'].append(ridge_meta)
    meta_models['scalers'].append(scaler_meta)

    # LightGBM meta
    meta_lgb_params = {
        'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.01,
        'num_leaves': 15, 'max_depth': 4, 'feature_fraction': 0.8,
        'bagging_fraction': 0.8, 'bagging_freq': 5, 'lambda_l1': 1.0,
        'lambda_l2': 1.0, 'verbosity': -1, 'seed': 42
    }

    train_meta_data = lgb.Dataset(X_tr_meta, label=y_tr_meta)
    val_meta_data = lgb.Dataset(X_val_meta, label=y_val_meta, reference=train_meta_data)

    lgb_meta = lgb.train(
        meta_lgb_params, train_meta_data, num_boost_round=1000,
        valid_sets=[val_meta_data],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
    )
    oof_meta_lgb[val_idx] = np.clip(lgb_meta.predict(X_val_meta), 0, 100)
    test_meta_lgb[:, fold-1] = np.clip(lgb_meta.predict(meta_test_enhanced), 0, 100)
    
    meta_models['lgb'].append(lgb_meta)

    # XGBoost meta
    dtrain_meta = xgb.DMatrix(X_tr_meta, label=y_tr_meta)
    dval_meta = xgb.DMatrix(X_val_meta, label=y_val_meta)
    dtest_meta = xgb.DMatrix(meta_test_enhanced)

    meta_xgb_params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse',
        'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.8,
        'colsample_bytree': 0.8, 'lambda': 2.0, 'alpha': 1.0, 'seed': 42
    }

    xgb_meta = xgb.train(
        meta_xgb_params, dtrain_meta, num_boost_round=1000,
        evals=[(dval_meta, 'eval')],
        early_stopping_rounds=50, verbose_eval=False
    )
    oof_meta_xgb[val_idx] = np.clip(xgb_meta.predict(dval_meta), 0, 100)
    test_meta_xgb[:, fold-1] = np.clip(xgb_meta.predict(dtest_meta), 0, 100)
    
    meta_models['xgb'].append(xgb_meta)

    print(f"✓")

print(f"\n{'='*80}")
print("LEVEL 2 META-LEARNER OOF SCORES")
print(f"{'='*80}")
print(f"Ridge Meta:    {np.sqrt(mean_squared_error(y_train, oof_meta_ridge)):.6f}")
print(f"LightGBM Meta: {np.sqrt(mean_squared_error(y_train, oof_meta_lgb)):.6f}")
print(f"XGBoost Meta:  {np.sqrt(mean_squared_error(y_train, oof_meta_xgb)):.6f}")

# Save meta-learner models
joblib.dump(meta_models, '../models/meta_models.pkl')
print("\n✓ Meta-learner models saved!")

# =============================================================================
# OPTIMAL ENSEMBLE
# =============================================================================
print(f"\n{'='*80}")
print("OPTIMIZING FINAL ENSEMBLE")
print(f"{'='*80}")

def ensemble_rmse_final(weights, *args):
    oof_preds, y_true = args
    weights = np.abs(weights) / np.sum(np.abs(weights))
    ensemble = sum(w * pred for w, pred in zip(weights, oof_preds))
    ensemble = np.clip(ensemble, 0, 100)
    return np.sqrt(mean_squared_error(y_true, ensemble))

# Combine all predictions
all_oof_preds = list(oof_predictions.values()) + [
    oof_meta_ridge, oof_meta_lgb, oof_meta_xgb
]

all_test_preds = list(test_pred_full.values()) + [
    test_meta_ridge.mean(axis=1),
    test_meta_lgb.mean(axis=1),
    test_meta_xgb.mean(axis=1)
]

# Optimize weights
initial_weights = np.ones(len(all_oof_preds)) / len(all_oof_preds)

result = minimize(
    ensemble_rmse_final, initial_weights, args=(all_oof_preds, y_train),
    method='Nelder-Mead',
    options={'maxiter': 2000, 'xatol': 1e-8, 'fatol': 1e-8}
)

optimal_weights = np.abs(result.x) / np.sum(np.abs(result.x))

print("\nOptimal weights:")
model_names = list(oof_predictions.keys()) + ['ridge_meta', 'lgb_meta', 'xgb_meta']
for name, weight in zip(model_names, optimal_weights):
    if weight > 0.01:
        print(f"  {name:15s}: {weight:.4f}")

# Create final ensemble
final_oof = sum(w * pred for w, pred in zip(optimal_weights, all_oof_preds))
final_oof = np.clip(final_oof, 0, 100)

final_test = sum(w * pred for w, pred in zip(optimal_weights, all_test_preds))
final_test = np.clip(final_test, 0, 100)

final_oof_rmse = np.sqrt(mean_squared_error(y_train, final_oof))

print(f"\n{'='*80}")
print("FINAL ENSEMBLE PERFORMANCE")
print(f"{'='*80}")
print(f"Final OOF RMSE: {final_oof_rmse:.6f}")

best_single = min([np.sqrt(mean_squared_error(y_train, pred)) for pred in all_oof_preds])
print(f"Best single model: {best_single:.6f}")
print(f"Improvement: {best_single - final_oof_rmse:.6f}")

# Save ensemble configuration
ensemble_config = {
    'weights': optimal_weights,
    'model_names': model_names,
    'weights_dict': {name: weight for name, weight in zip(model_names, optimal_weights)},
    'final_oof_rmse': final_oof_rmse
}
joblib.dump(ensemble_config, '../models/ensemble_config.pkl')
print("\n✓ Ensemble configuration saved!")

# =============================================================================
# POST-PROCESSING & CALIBRATION
# =============================================================================
print(f"\n{'='*80}")
print("POST-PROCESSING & CALIBRATION")
print(f"{'='*80}")

def calibrate_predictions(train_preds, train_true, test_preds):
    """Apply isotonic regression calibration"""
    iso_reg = IsotonicRegression(out_of_bounds='clip')
    iso_reg.fit(train_preds, train_true)
    test_calibrated = iso_reg.predict(test_preds)
    return np.clip(test_calibrated, 0, 100)

# Apply calibration
final_test_calibrated = calibrate_predictions(final_oof, y_train, final_test)

# Check if calibration improves OOF score
print("Testing calibration effectiveness...")
final_oof_calibrated = calibrate_predictions(
    final_oof[::2], y_train[::2],  # Train on even indices
    final_oof[1::2]  # Test on odd indices
)

# Create full calibrated OOF
calibrated_oof = final_oof.copy()
calibrated_oof[1::2] = final_oof_calibrated

calibrated_rmse = np.sqrt(mean_squared_error(y_train, calibrated_oof))
print(f"Original OOF RMSE:   {final_oof_rmse:.6f}")
print(f"Calibrated OOF RMSE: {calibrated_rmse:.6f}")

if calibrated_rmse < final_oof_rmse:
    print("✓ Using calibrated predictions (improvement detected)")
    final_submission = final_test_calibrated
    final_rmse = calibrated_rmse
else:
    print("✓ Using original predictions (no improvement from calibration)")
    final_submission = final_test
    final_rmse = final_oof_rmse

# =============================================================================
# FINAL STATISTICS & SUBMISSION
# =============================================================================
print(f"\n{'='*80}")
print("FINAL STATISTICS")
print(f"{'='*80}")

print(f"\nPrediction Statistics:")
print(f"  Train target - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}, Min: {y_train.min():.2f}, Max: {y_train.max():.2f}")
print(f"  OOF preds    - Mean: {final_oof.mean():.2f}, Std: {final_oof.std():.2f}, Min: {final_oof.min():.2f}, Max: {final_oof.max():.2f}")
print(f"  Test preds   - Mean: {final_submission.mean():.2f}, Std: {final_submission.std():.2f}, Min: {final_submission.min():.2f}, Max: {final_submission.max():.2f}")

print(f"\n{'='*80}")
print("FINAL PERFORMANCE")
print(f"{'='*80}")
print(f"✓ Final OOF RMSE: {final_rmse:.6f}")
print(f"✓ Expected LB Score: ~{final_rmse:.4f} (±0.002)")
print(f"\n🎯 Target beaten: {final_rmse < 8.54414}")

# Create submission
os.makedirs('../results', exist_ok=True)

submission = pd.DataFrame({
    'id': test_df['id'],
    'exam_score': final_submission
})

submission.to_csv('../results/submission.csv', index=False)
print(f"\n✓ Submission saved to ../results/submission.csv")

# Save OOF predictions
train_df = pd.read_csv("../data/train.csv")
oof_df = pd.DataFrame({
    'id': train_df['id'],
    'exam_score': y_train,
    'prediction': final_oof
})
oof_df.to_csv('../results/oof_predictions.csv', index=False)
print("✓ OOF predictions saved to ../results/oof_predictions.csv")

# Save individual model OOF predictions for analysis
oof_detailed = pd.DataFrame({
    'id': train_df['id'],
    'exam_score': y_train
})
for model_name, oof in oof_predictions.items():
    oof_detailed[f'oof_{model_name}'] = oof

oof_detailed['oof_ridge_meta'] = oof_meta_ridge
oof_detailed['oof_lgb_meta'] = oof_meta_lgb
oof_detailed['oof_xgb_meta'] = oof_meta_xgb
oof_detailed['oof_final_ensemble'] = final_oof

oof_detailed.to_csv('../results/oof_predictions_detailed.csv', index=False)
print("✓ Detailed OOF predictions saved to ../results/oof_predictions_detailed.csv")

# Save model comparison
model_comparison = pd.DataFrame({
    'model': model_names,
    'oof_rmse': [np.sqrt(mean_squared_error(y_train, pred)) for pred in all_oof_preds],
    'ensemble_weight': optimal_weights
}).sort_values('oof_rmse')

model_comparison.to_csv('../results/model_comparison.csv', index=False)
print("✓ Model comparison saved to ../results/model_comparison.csv")

# Save best iterations info
best_iterations_df = pd.DataFrame({
    'model': ['XGBoost', 'LightGBM', 'CatBoost'],
    'avg_best_iteration': [avg_best_iter_xgb, avg_best_iter_lgb, avg_best_iter_cat],
    'iterations_per_fold': [
        str(best_iterations['xgb']),
        str(best_iterations['lgb']),
        str(best_iterations['cat'])
    ]
})
best_iterations_df.to_csv('../results/best_iterations.csv', index=False)
print("✓ Best iterations saved to ../results/best_iterations.csv")

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print(f"\n{'='*80}")
print("TRAINING PIPELINE COMPLETE!")
print(f"{'='*80}")
print(f"\n📊 PERFORMANCE SUMMARY")
print(f"{'='*60}")
print(f"Final OOF RMSE:        {final_rmse:.6f}")
print(f"Best Single Model:     {best_single:.6f}")
print(f"Ensemble Improvement:  {best_single - final_rmse:.6f}")
print(f"\n🏆 TOP 3 MODELS BY OOF RMSE:")
top_3 = model_comparison.head(3)
for idx, row in top_3.iterrows():
    print(f"  {row['model']:15s}: {row['oof_rmse']:.6f} (weight: {row['ensemble_weight']:.4f})")

print(f"\n📁 SAVED FILES")
print(f"{'='*60}")
print(f"Models (../models/):")
print(f"  ✓ ridge_full.pkl")
print(f"  ✓ elastic_full.pkl")
print(f"  ✓ xgb_full.pkl")
print(f"  ✓ lgb_full.pkl")
print(f"  ✓ cat_full.pkl")
print(f"  ✓ et_full.pkl")
print(f"  ✓ gbr_full.pkl")
print(f"  ✓ svr_full.pkl")
print(f"  ✓ meta_models.pkl (Ridge + LightGBM + XGBoost meta-learners)")
print(f"  ✓ ensemble_config.pkl")

print(f"\nResults (../results/):")
print(f"  ✓ submission.csv (READY FOR KAGGLE)")
print(f"  ✓ oof_predictions.csv")
print(f"  ✓ oof_predictions_detailed.csv")
print(f"  ✓ model_comparison.csv")
print(f"  ✓ best_iterations.csv")

print(f"\nOptimization Results (../results/optimization/):")
print(f"  ✓ lightgbm_params.pkl + history + study")
print(f"  ✓ xgboost_params.pkl + history + study")
print(f"  ✓ catboost_params.pkl + history + study")
print(f"  ✓ extratrees_params.pkl + history + study")
print(f"  ✓ gradientboosting_params.pkl + history + study")
print(f"  ✓ ridge_params.pkl + history + study")
print(f"  ✓ elasticnet_params.pkl + history + study")
print(f"  ✓ svr_params.pkl + history + study")

print(f"\n{'='*80}")
print(f"🎉 ALL DONE! Submission ready for Kaggle!")
print(f"{'='*80}")
print(f"\n💡 Next Steps:")
print(f"  1. Review model_comparison.csv to see individual model performance")
print(f"  2. Check oof_predictions_detailed.csv for model-by-model predictions")
print(f"  3. Upload submission.csv to Kaggle")
print(f"  4. If needed, adjust ensemble weights in ensemble_config.pkl")
print(f"\n✨ Good luck with your submission!")
print(f"{'='*80}")